In [1]:
import gensim
import gensim.downloader as api
import numpy as np
import re
import csv
import pandas as pd
import pprint
import string
import nltk
import sys

from IPython.display import HTML
from nltk.corpus import wordnet 
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors


In [2]:
pathToDatasets = '../datasets/'
pathToDataScripts = '../datasets/scripts/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'
word_vectors = api.load("glove-wiki-gigaword-100")
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
sys.path.insert(0, pathToDataScripts)
from cleanDataset import tokenize_words 



senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;


NUMBER_OF_ALTERNATIVES = 7
TWEET_START = 50
NUM_OF_TWEETS = 25

punctuation = r"\"#$%&'()+-/:;<=>?@[\]*^_`{|}~"

/home/ubuntu/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

## Utility Code

In [5]:
def cleanAndTokenizeText(text):
    text = text.lower()
    newString = ""
    for char in text:
        if char not in punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

def listReplacements(word):
    if(word not in vocabulary):
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=NUMBER_OF_ALTERNATIVES)]
    antonyms = getAntonymsOfWords(word)
    if(antonyms != []):
        print("For word [{1}]: antonyms - {0}".format(antonyms, word))
        possibleReplacements.extend(antonyms)
    return possibleReplacements

def getPOSTags(tweet):
    tags = nltk.pos_tag(tweet)
    return(tags)

def getAntonymsOfWords(word):
    if (word not in vocabulary):
        return []
    listOfAntonyms = []
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            anton = l.antonyms()
            if(anton != []):
                listOfAntonyms.append(anton[0].name())
    return list(set(listOfAntonyms))

## NLP Utility Code

In [6]:
def posApprovedReplacements(alternativeWords, userTokens, indexOfToken):
    if(alternativeWords == []):
        print("--- No alternative words! ---")
        return []
    tempTokens = userTokens[:]
    truePOSTokens = getPOSTags(tempTokens)
    validWords = []
    
    mainTag = truePOSTokens[indexOfToken][1]
    mainWord = userTokens[indexOfToken]
    
    for ind,word in enumerate(alternativeWords):
        tempTokens[indexOfToken] = word
        posTags = getPOSTags(tempTokens)
        newTag = (posTags[indexOfToken])[1]

        if(str(newTag) == str(mainTag)):
            print("Word {0}[{1}] replaced with {2}[{3}]".format(mainWord, mainTag, word,newTag))
            validWords.append(word)
    return validWords
        

In [7]:
def getAlternativeSentences(tweet, sentimentOfTweet):
    userInputTokens = cleanAndTokenizeText(tweet)
    
   
    alternativeStrings = []
    for ind,word in enumerate(userInputTokens):
        
        score = senty.polarity_scores(word)['compound'] # get the aggregated score!
        newUserTokens = userInputTokens[:]
        
        if(score != 0.0):
            replacements = listReplacements(word)       
            replacements = posApprovedReplacements(replacements[:], newUserTokens[:], ind)
            
            if(replacements == []):
                print("--- No pos approved words! ---")
                continue          
            for newWord in replacements:
                
                htmlFriendlyTokens = newUserTokens[:]
                newUserTokens[ind] = newWord
                htmlFriendlyTokens[ind] = cstr(" <i>[{0}]</i> ".format(newWord), 'blue');
                newString = ' '.join(newUserTokens)
                print(newString)
                sentimentOfNewString = senty.polarity_scores(newString)['compound']
                
#                 if(sentimentOfNewString == 0):
#                     continue
                
                alternativeStrings.append(' '.join(htmlFriendlyTokens))
    return alternativeStrings;

## Test Cells

In [8]:
specificWord = "good"
def testOneWord(word = ""):
    if(word == ""):
        return
    else:
        print(getAntonymsOfWords(word))

testOneWord(specificWord)

['evilness', 'evil', 'bad', 'badness', 'ill']


In [9]:
specificString = ""
def specificString(textString=""):
    if(textString == "" or textString == None):
        return
    mainSentiment = senty.polarity_scores(textString)['compound']
    if(mainSentiment == 0):
        print(" Sentiment of String shows completely neutral ");
        return
    else:
        print("{0}: {1}\n".format(textString, mainSentiment))
        newStrings = getAlternativeSentences(textString, mainSentiment)
        newStrings.append("this is a very [good] phrase that needs to be understood by everyone !")
        if(newStrings == [] or newStrings == None):
            print("--- No new strings generated ---")
            return
        for newStr in newStrings:
            sentimentOfNewString = senty.polarity_scores(newStr)['compound']
            if(sentimentOfNewString == mainSentiment or sentimentOfNewString == 0):
                display(HTML(cstr("{0}: {1}".format(newStr,sentimentOfNewString),'DarkGray')))
            elif(sentimentOfNewString > mainSentiment):
                display(HTML(cstr("{0}: {1}".format(newStr,sentimentOfNewString),'green')))
            else:
                display(HTML(cstr("{0}: {1}".format(newStr,sentimentOfNewString),'red')))

                
                

# specificString = "I bought this for my husband who plays the piano.  \
# He is having a wonderful time playing these old hymns.  \
# The music  is at times hard to read because we think the book was \
# published for singing from more than playing from.  Great purchase though!"
specificString("This is a very good phrase that needs to be understood by everyone!")
specificString("This is a pretty good phrase that needs to be understood by everyone!")
specificString("This is a kinda good phrase that needs to be understood by everyone!")
specificString("This is a good phrase that needs to be understood by everyone!")
specificString("MVB has unpleasant weather. That's why I got a fan!")


This is a very good phrase that needs to be understood by everyone!: 0.54

For word [good]: antonyms - ['evilness', 'evil', 'bad', 'badness', 'ill']
Word good[JJ] replaced with better[JJ]
Word good[JJ] replaced with sure[JJ]
Word good[JJ] replaced with evilness[JJ]
Word good[JJ] replaced with evil[JJ]
Word good[JJ] replaced with bad[JJ]
Word good[JJ] replaced with badness[JJ]
Word good[JJ] replaced with ill[JJ]
this is a very better phrase that needs to be understood by everyone !
this is a very sure phrase that needs to be understood by everyone !
this is a very evilness phrase that needs to be understood by everyone !
this is a very evil phrase that needs to be understood by everyone !
this is a very bad phrase that needs to be understood by everyone !
this is a very badness phrase that needs to be understood by everyone !
this is a very ill phrase that needs to be understood by everyone !


This is a pretty good phrase that needs to be understood by everyone!: 0.75

For word [pretty]: antonyms - ['unreasonably', 'immoderately']
Word pretty[RB] replaced with quite[RB]
Word pretty[RB] replaced with really[RB]
Word pretty[RB] replaced with very[RB]
Word pretty[RB] replaced with too[RB]
Word pretty[RB] replaced with incredibly[RB]
Word pretty[RB] replaced with unreasonably[RB]
Word pretty[RB] replaced with immoderately[RB]
this is a quite good phrase that needs to be understood by everyone !
this is a really good phrase that needs to be understood by everyone !
this is a very good phrase that needs to be understood by everyone !
this is a too good phrase that needs to be understood by everyone !
this is a incredibly good phrase that needs to be understood by everyone !
this is a unreasonably good phrase that needs to be understood by everyone !
this is a immoderately good phrase that needs to be understood by everyone !
For word [good]: antonyms - ['evilness', 'evil', 'bad', 

This is a kinda good phrase that needs to be understood by everyone!: 0.4402

For word [good]: antonyms - ['evilness', 'evil', 'bad', 'badness', 'ill']
Word good[JJ] replaced with sure[JJ]
Word good[JJ] replaced with evil[JJ]
Word good[JJ] replaced with bad[JJ]
this is a kinda sure phrase that needs to be understood by everyone !
this is a kinda evil phrase that needs to be understood by everyone !
this is a kinda bad phrase that needs to be understood by everyone !


This is a good phrase that needs to be understood by everyone!: 0.4926

For word [good]: antonyms - ['evilness', 'evil', 'bad', 'badness', 'ill']
Word good[JJ] replaced with sure[JJ]
Word good[JJ] replaced with evilness[JJ]
Word good[JJ] replaced with evil[JJ]
Word good[JJ] replaced with bad[JJ]
Word good[JJ] replaced with badness[JJ]
Word good[JJ] replaced with ill[JJ]
this is a sure phrase that needs to be understood by everyone !
this is a evilness phrase that needs to be understood by everyone !
this is a evil phrase that needs to be understood by everyone !
this is a bad phrase that needs to be understood by everyone !
this is a badness phrase that needs to be understood by everyone !
this is a ill phrase that needs to be understood by everyone !


MVB has unpleasant weather. That's why I got a fan!: -0.2714

For word [unpleasant]: antonyms - ['pleasant']
Word unpleasant[JJ] replaced with uncomfortable[JJ]
Word unpleasant[JJ] replaced with painful[JJ]
Word unpleasant[JJ] replaced with disagreeable[JJ]
Word unpleasant[JJ] replaced with pleasant[JJ]
mvb has uncomfortable weather . thats why i got a fan !
mvb has painful weather . thats why i got a fan !
mvb has disagreeable weather . thats why i got a fan !
mvb has pleasant weather . thats why i got a fan !
Word fan[NN] replaced with star[NN]
Word fan[NN] replaced with player[NN]
Word fan[NN] replaced with crowd[NN]
Word fan[NN] replaced with fame[NN]
Word fan[NN] replaced with game[NN]
mvb has unpleasant weather . thats why i got a star !
mvb has unpleasant weather . thats why i got a player !
mvb has unpleasant weather . thats why i got a crowd !
mvb has unpleasant weather . thats why i got a fame !
mvb has unpleasant weather . thats why i got a game !


## Main Cells

In [10]:
def shiftSentiment(actualTweet,mainSentiment, alternateTweets, positive=True):

    happiestTweet = ""
    saddestTweet = ""
    happiestScore = -sys.maxsize - 1
    saddestScore = sys.maxsize
    correctTweets = []
    
    for tweet in alternateTweets:
        newSenty = senty.polarity_scores(tweet)['compound']
        
        if(newSenty < saddestScore):
            saddestTweet = tweet
            saddestScore = newSenty
        if(newSenty > happiestScore):
            happiestTweet = tweet
            happiestScore = newSenty
            
        if(positive):
            if(newSenty > mainSentiment):
                correctTweets.append(tweet)
                continue
            
            elif(newSenty < mainSentiment):
                continue
                # Grab happiest tweet and if it's not "", then generate more happy tweets from it
        if(positive == False):
            if(newSenty < mainSentiment):
                correctTweets.append(tweet)
                continue
            
            elif(newSenty > mainSentiment):
                continue
                # grab happiest tweet, and if it's not "", then generate more happy tweets from it
                
    if(correctTweets == []):
        print("\n\nNo tweets found when trying to do Positive={}\n\n".format(positive))
    return correctTweets;
        

In [11]:
def printStrings(newStrings, mainSentiment):
    for alteredTweet in newStrings:
            sentimentOfNewString = senty.polarity_scores(alteredTweet)['compound']
            if(sentimentOfNewString == mainSentiment or sentimentOfNewString == 0):
                display(HTML(cstr("{0}: {1}".format(alteredTweet,sentimentOfNewString),'DarkGray')))
            elif(sentimentOfNewString > mainSentiment):
                display(HTML(cstr("{0}: {1}".format(alteredTweet,sentimentOfNewString),'green')))
            else:
                display(HTML(cstr("{0}: {1}".format(alteredTweet,sentimentOfNewString),'red')))

In [12]:
def runThroughTweets():

    tweets_df = pd.read_csv( pathToDatasets + 'cleanedTweets.csv' , nrows=NUM_OF_TWEETS, skiprows=TWEET_START)

    tweets = tweets_df.values

    
    for counter,tweet in enumerate(tweets):
        tweet = tweet[0]
        
        mainSentiment = senty.polarity_scores(tweet)['compound']
        if(mainSentiment == 0):
            continue
        print("\nNUM({2}) {0}:{1}\n".format(tweet,mainSentiment, counter))       
        alternateTweets = getAlternativeSentences(tweet, mainSentiment)
        if(alternateTweets == [] or alternateTweets == None):
            print("--- No new strings generated ---\n\n")
            continue
        newStrings = shiftSentiment(tweet, mainSentiment, alternateTweets, True);
        printStrings(newStrings, mainSentiment)
        newStringsAgain = shiftSentiment(tweet, mainSentiment, alternateTweets, False);
        printStrings(newStringsAgain, mainSentiment)
    
runThroughTweets()


NUM(0) broadband plan a massive broken promise  via  still waiting for broadband we are :-0.2023

For word [broken]: antonyms - ['keep', 'unbroken', 'conform_to', 'make', 'promote', 'repair']
Word broken[NN] replaced with breaking[NN]
Word broken[NN] replaced with broke[NN]
Word broken[NN] replaced with apart[NN]
Word broken[NN] replaced with neck[NN]
Word broken[NN] replaced with keep[NN]
Word broken[NN] replaced with conform_to[NN]
Word broken[NN] replaced with make[NN]
Word broken[NN] replaced with promote[NN]
Word broken[NN] replaced with repair[NN]
broadband plan a massive breaking promise via still waiting for broadband we are
broadband plan a massive broke promise via still waiting for broadband we are
broadband plan a massive apart promise via still waiting for broadband we are
broadband plan a massive neck promise via still waiting for broadband we are
broadband plan a massive keep promise via still waiting for broadband we are
broadband plan a massive conform_to promise via 


NUM(1)  wow tons of replies from you may have to unfollow so i can see my friends tweets you are scrolling the feed a lot. :0.7845

Word wow[JJ] replaced with c'mon[JJ]
Word wow[JJ] replaced with hey[JJ]
Word wow[JJ] replaced with gosh[JJ]
Word wow[JJ] replaced with whew[JJ]
c'mon tons of replies from you may have to unfollow so i can see my friends tweets you are scrolling the feed a lot .
hey tons of replies from you may have to unfollow so i can see my friends tweets you are scrolling the feed a lot .
gosh tons of replies from you may have to unfollow so i can see my friends tweets you are scrolling the feed a lot .
whew tons of replies from you may have to unfollow so i can see my friends tweets you are scrolling the feed a lot .
For word [friends]: antonyms - ['stranger', 'foe']
Word friends[NNS] replaced with parents[NNS]
Word friends[NNS] replaced with relatives[NNS]
Word friends[NNS] replaced with others[NNS]
wow tons of replies from you may have to unfollow so i can see my pa


NUM(3) put vacation photos online a few yrs ago. pc crashed and now i forget the name of the site. :-0.2263

For word [forget]: antonyms - ['mind', 'remember']
Word forget[VBP] replaced with remember[VBP]
Word forget[VBP] replaced with tell[VBP]
Word forget[VBP] replaced with imagine[VBP]
Word forget[VBP] replaced with know[VBP]
Word forget[VBP] replaced with mind[VBP]
Word forget[VBP] replaced with remember[VBP]
put vacation photos online a few yrs ago . pc crashed and now i remember the name of the site .
put vacation photos online a few yrs ago . pc crashed and now i tell the name of the site .
put vacation photos online a few yrs ago . pc crashed and now i imagine the name of the site .
put vacation photos online a few yrs ago . pc crashed and now i know the name of the site .
put vacation photos online a few yrs ago . pc crashed and now i mind the name of the site .
put vacation photos online a few yrs ago . pc crashed and now i remember the name of the site .




No tweets found when trying to do Positive=False



NUM(4) i need a hug :0.4767

Word hug[NN] replaced with hugs[NN]
Word hug[NN] replaced with kiss[NN]
Word hug[NN] replaced with hugged[NN]
Word hug[NN] replaced with goodbye[NN]
i need a hugs
i need a kiss
i need a hugged
i need a goodbye


No tweets found when trying to do Positive=True





NUM(5)  not sure what they are only that they are pos! as much as i want to i dont think can trade away company assets sorry andy! :-0.2134

For word [sure]: antonyms - ['uncertain', 'unsure']
Word sure[JJ] replaced with uncertain[JJ]
Word sure[JJ] replaced with unsure[JJ]
not uncertain what they are only that they are pos ! as much as i want to i dont think can trade away company assets sorry andy !
not unsure what they are only that they are pos ! as much as i want to i dont think can trade away company assets sorry andy !
Word want[VBP] replaced with do[VBP]
Word want[VBP] replaced with know[VBP]
Word want[VBP] replaced with get[VBP]
not sure what they are only that they are pos ! as much as i do to i dont think can trade away company assets sorry andy !
not sure what they are only that they are pos ! as much as i know to i dont think can trade away company assets sorry andy !
not sure what they are only that they are pos ! as much as i get to i dont think can trade away company as

...


NUM(17) mo jobs no money.  how in the hell is min wage here 4 fn clams an hour:-0.7783

For word [no]: antonyms - ['all', 'yes']
Word no[DT] replaced with any[DT]
Word no[DT] replaced with all[DT]
mo jobs any money . how in the hell is min wage here 4 fn clams an hour
mo jobs all money . how in the hell is min wage here 4 fn clams an hour
For word [hell]: antonyms - ['Heaven']
Word hell[NN] replaced with heaven[NN]
Word hell[NN] replaced with crazy[NN]
mo jobs no money . how in the heaven is min wage here 4 fn clams an hour
mo jobs no money . how in the crazy is min wage here 4 fn clams an hour




No tweets found when trying to do Positive=False



NUM(19)  agreed i saw the failwhale allllll day today. :0.2732

For word [agreed]: antonyms - ['disagree']
--- No pos approved words! ---
--- No new strings generated ---



NUM(20)  oh! haha... dude i dont really look at em unless someone says hey i added you. sorry  i am so terrible at that. i need a pop up!:-0.6459

Word haha[NN] replaced with prude[NN]
Word haha[NN] replaced with porpoise[NN]
Word haha[NN] replaced with será[NN]
Word haha[NN] replaced with clavaria[NN]
Word haha[NN] replaced with thunderer[NN]
oh ! prude ... dude i dont really look at em unless someone says hey i added you . sorry i am so terrible at that . i need a pop up !
oh ! porpoise ... dude i dont really look at em unless someone says hey i added you . sorry i am so terrible at that . i need a pop up !
oh ! será ... dude i dont really look at em unless someone says hey i added you . sorry i am so terrible at that . i need a pop up !
oh ! clavaria ... dude



No tweets found when trying to do Positive=False



NUM(21)  i am sure you are right...    i need to start working out with you and the nikster... or jared at least!:0.3802

For word [sure]: antonyms - ['uncertain', 'unsure']
Word sure[JJ] replaced with uncertain[JJ]
i am uncertain you are right ... i need to start working out with you and the nikster ... or jared at least !


No tweets found when trying to do Positive=True





NUM(22) i really hate how people diss my bands!  trace is clearly not ugly!:0.2484

For word [hate]: antonyms - ['love']
Word hate[VB] replaced with racist[VB]
Word hate[VB] replaced with racism[VB]
Word hate[VB] replaced with fear[VB]
Word hate[VB] replaced with bigotry[VB]
Word hate[VB] replaced with love[VB]
i really racist how people diss my bands ! trace is clearly not ugly !
i really racism how people diss my bands ! trace is clearly not ugly !
i really fear how people diss my bands ! trace is clearly not ugly !
i really bigotry how people diss my bands ! trace is clearly not ugly !
i really love how people diss my bands ! trace is clearly not ugly !
For word [clearly]: antonyms - ['unintelligibly']
Word clearly[RB] replaced with obviously[RB]
Word clearly[RB] replaced with certainly[RB]
Word clearly[RB] replaced with indeed[RB]
Word clearly[RB] replaced with nonetheless[RB]
Word clearly[RB] replaced with unintelligibly[RB]
i really hate how people diss my bands ! trace is obvio


NUM(23) gym attire today was puma singlet adidas shorts.......and black business socks and leather shoes  lucky did not run into any cute girls.:0.7003

For word [lucky]: antonyms - ['unlucky']
Word lucky[RB] replaced with unlucky[RB]
Word lucky[RB] replaced with maybe[RB]
Word lucky[RB] replaced with unlucky[RB]
gym attire today was puma singlet adidas shorts ... ... .and black business socks and leather shoes unlucky did not run into any cute girls .
gym attire today was puma singlet adidas shorts ... ... .and black business socks and leather shoes maybe did not run into any cute girls .
gym attire today was puma singlet adidas shorts ... ... .and black business socks and leather shoes unlucky did not run into any cute girls .
Word cute[JJ] replaced with adorable[JJ]
Word cute[JJ] replaced with goofy[JJ]
Word cute[JJ] replaced with sexy[JJ]
Word cute[JJ] replaced with cuddly[JJ]
Word cute[JJ] replaced with funny[JJ]
Word cute[JJ] replaced with perky[JJ]
Word cute[JJ] replaced with n